# Installs 

In [1]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.0 MB/s 


In [2]:
pip install transtab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 20.2 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 119 kB 67.4 MB/s 
     |████████████████████████████████| 76 kB 6.0 MB/s 
     |████████████████████████████████| 7.6 MB 55.1 MB/s 
     |████████████████████████████████| 182 kB 71.9 MB/s 
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137412 sha256=05274a13ac6629447b2724643c277f04d099eb8bffb5f7709163e15da1e1b123
  Stored in directory: /root/.cache/pip/wheels/6a/20/88/cf4ac86aa18e2cd647ed16ebe274a5dacee9d0075fa02af250
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11730 sha256=bfd3e925ca0dd3524127a00c005db0e816fc89bb96dcd40dd94acbf24116867d
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml liac-arff

# Imports

In [3]:
import os
import pdb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import openml
from loguru import logger
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import transtab

# Helpers

In [4]:
def encode(df_train,df_test,target,cat_cols):
  y = df_train[target].values
  X=df_train.drop([target], axis=1)
  y_test = df_test[target].values
  X_test=df_test.drop([target], axis=1)
  encoder = ce.LeaveOneOutEncoder()
  train = encoder.fit_transform(X[cat_cols], y)
  test=encoder.transform(X_test[cat_cols])
  cols=list(df_train)
  cols2=list(train)
  for col in cols:
      if col not in cols2:
          train[col]=df_train[col]
  train[target]=y

  cols2=list(test)
  for col in cols:
      if col not in cols2:
          test[col]=df_test[col]
  test[target]=y_test
    
  return train,test

In [5]:
def normalize(df_train,df_test,target,cat_cols):
  cols=list(df_train)
  cat_cols.append(target)
  cols_en=[]
  for i in cols:
    if i not in cat_cols:
      cols_en.append(i)

  X_train=df_train.drop(cat_cols,axis=1).values
  X_train =quantile_transform(X_train, random_state=0, copy=True,n_quantiles=int(len(X_train)/4))
  df_train_scaled=pd.DataFrame(X_train,columns=cols_en)

  cols=list(df_train)
  for col in cols:
    if col not in cols_en:
      df_train_scaled[col]=df_train[col]
  
  X_test=df_test.drop(cat_cols,axis=1).values
  X_test =quantile_transform(X_test, random_state=0, copy=True,n_quantiles=int(len(X_test)/4))
  df_test_scaled=pd.DataFrame(X_test,columns=cols_en)

  cols=list(df_test)
  for col in cols:
    if col not in cols_en:
      df_test_scaled[col]=df_test[col]
  
  return df_train_scaled, df_test_scaled

In [6]:
#CREDITS: https://github.com/RyanWangZf/transtab/blob/main/transtab/dataset.py


def load_test_data(df, dataset_config=None, encode_cat=False, seed=123):
    y = df['Class']
    X = df.drop(['Class'],axis=1)
    all_cols = [col.lower() for col in X.columns.tolist()]
    print(all_cols)
    X.columns = all_cols
    attribute_names = all_cols
    num_cols_ = dataset_config['num']
    bin_cols_ = dataset_config['bin']
    cat_cols_ = dataset_config['cat']
    cat_cols=[col.lower() for col in cat_cols_]
    bin_cols=[col.lower() for col in bin_cols_]
    num_cols=[col.lower() for col in num_cols_]     
    X = X[bin_cols + num_cols + cat_cols]
    print('# data: {}, # feat: {}, # cate: {},  # bin: {}, # numerical: {}, pos rate: {:.2f}'.format(len(X), len(attribute_names), len(cat_cols), len(bin_cols), len(num_cols), (y==1).sum()/len(y)))
    return (X,y)

In [7]:
#CREDITS: https://github.com/RyanWangZf/transtab/blob/main/transtab/dataset.py

def load_data(df=None, dataset_config=None, encode_cat=False, data_cut=None, seed=123):
  return load_single_data(df=df, dataset_config=dataset_config, encode_cat=encode_cat, data_cut=data_cut, seed=seed)

def load_single_data(df=None, dataset_config=None, encode_cat=False, data_cut=None, seed=123):
    y = df['Class']
    X = df.drop(['Class'],axis=1)
    all_cols = [col.lower() for col in X.columns.tolist()]
    X.columns = all_cols
    attribute_names = all_cols
    num_cols_ = dataset_config['num']
    bin_cols_ = dataset_config['bin']
    cat_cols_ = dataset_config['cat']
    cat_cols=[col.lower() for col in cat_cols_]
    print(cat_cols)
    bin_cols=[col.lower() for col in bin_cols_]
    print(bin_cols)
    num_cols=[col.lower() for col in num_cols_]
    print(num_cols)
    X = X[bin_cols + num_cols + cat_cols]
    train_dataset, val_dataset, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y,random_state=seed, shuffle=True)
    print('# data: {}, # feat: {}, # cate: {},  # bin: {}, # numerical: {}, pos rate: {:.2f}'.format(len(X), len(attribute_names), len(cat_cols), len(bin_cols), len(num_cols), (y==1).sum()/len(y)))
    return (X,y), (train_dataset,y_train),(val_dataset,y_val), cat_cols, num_cols, bin_cols

In [8]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
def metrics(cm):
        TN, FP, FN, TP = cm.ravel()
        precision =(TP)/(TP+FP)
        accuracy_model  =(TP+TN)/(TP+TN+FP+FN)
        recall_score  =(TP)/(TP+FN)
        specificity_value =(TN)/(TN + FP)
        False_positive_rate =(FP)/(FP+TN)
        False_negative_rate =(FN)/(FN+TP)
        f1_score =2*(( precision * recall_score)/( precision + recall_score))
        PPV=(TP)/(TP+FP)
        NPV=TN / (TN + FN) 
        print("PPV: ",PPV*100)
        print("NPV: ",NPV*100)
        print("Precision value of the model: ",precision*100)
        print("Accuracy of the model: ",accuracy_model*100)
        print("Recall of the model: ",recall_score*100)
        print("Specificity of the model: ",specificity_value*100)
        print("F1 Score of the model: ",f1_score*100)

In [15]:
def getCols(df):
  all=list(df)
  df2=df.dropna()
  all.remove('Class')
  num_cols=[]
  for i in all:
    if len(np.unique(df2[i]))>50:
      num_cols.append(i)
  bin_cols=[]
  bin_cols = [col for col in df2 if np.isin(df2[col].unique(), [0, 1]).all()]
  cat_cols=[]
  for i in all:
    if i not in num_cols and i not in bin_cols and i!='Class':
      cat_cols.append(i)
  if 'Class' in bin_cols:
    bin_cols.remove('Class')
  return [num_cols,cat_cols,bin_cols]

# Machine Learning Models

## MIMIC-III Extracted Delirium Dataset

#### Encoded Data

In [11]:
df=pd.read_csv('/content/delirium_icu_tr_mimic_encoded.csv')

In [12]:
df.head()

,ALT,AST,Admission Weight (Kg),Albumin,Anion gap,Arterial Base Excess,Arterial Blood Pressure mean,Arterial CO2(Calc),Arterial PaCO2,Arterial pH,...,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,GENDER,AGE,Class
0,25.666667,77.000000,73.777055,2.950000,13.226413,0.534483,74.230570,26.051724,40.422414,7.400342,...,0,2,3,17,5,2,25,1,63.178082,0
1,27.750000,41.000000,79.876167,1.925000,13.167512,3.260000,59.571885,29.640000,45.020000,7.410200,...,2,3,3,17,3,2,25,1,53.805479,0
2,1084.977273,2380.916996,72.008629,2.715033,12.880346,0.535714,87.937727,25.482143,38.511364,7.412403,...,1,2,2,17,12,4,25,1,50.260274,1
3,25.500000,36.000000,78.602700,3.700000,13.199486,1.088235,78.837134,26.058824,37.323529,7.442553,...,1,0,3,29,12,2,25,1,56.493151,0
4,317.207407,812.911111,76.556766,2.777419,13.723353,-0.797357,100.200931,23.159292,33.132159,7.429607,...,1,3,3,17,3,3,25,0,41.293151,0


In [16]:
num_cols,cat_cols,bin_cols=getCols(df)

In [17]:
dataset_config={'bin': bin_cols,'cat':cat_cols,'num':num_cols}
# load dataset by specifying dataset name
allset, trainset, valset, cat_cols, num_cols, bin_cols= load_data(df,dataset_config=dataset_config)

['admission_type', 'admission_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity']
['gender']
['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2(calc)', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood - calc)', 'hemoglobin', 'high resp. rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak insp. pressure', 'phosphorous', 'plateau pressure', 'platelet count', 'potassium (wh

In [18]:
test=pd.read_csv('/content/delirium_icu_ts_mimic_encoded.csv')
testset=load_test_data(test,dataset_config=dataset_config)

['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2(calc)', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood - calc)', 'hemoglobin', 'high resp. rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak insp. pressure', 'phosphorous', 'plateau pressure', 'platelet count', 'potassium (whole blood)', 'pressure support', 'prothrombin time', 'resp rate (spont)', 'respiratory rate', 'respiratory rate set', 's

In [19]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 59, test auc: 0.845573
EarlyStopping counter: 29 out of 30
epoch: 59, train loss: 24.3352, lr: 0.000100, spent: 250.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:26:35.747 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:26:35.774 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 60, test auc: 0.855588
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:26:35.983 | INFO     | transtab.trainer:train:137 - training complete, cost 254.9 secs.


In [20]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.5230643694379
PPV:  76.99680511182109
NPV:  85.75418994413407
Precision value of the model:  76.99680511182109
Accuracy of the model:  83.09037900874635
Recall of the model:  70.26239067055393
Specificity of the model:  89.50437317784257
F1 Score of the model:  73.47560975609755


In [21]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.693575
epoch: 0, train loss: 62.1312, lr: 0.000100, spent: 4.0 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.725742
epoch: 1, train loss: 57.5613, lr: 0.000100, spent: 8.0 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 52, test auc: 0.874230
EarlyStopping counter: 29 out of 30
epoch: 52, train loss: 25.5944, lr: 0.000100, spent: 213.1 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:30:13.495 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:30:13.516 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 53, test auc: 0.869406
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:30:13.764 | INFO     | transtab.trainer:train:137 - training complete, cost 217.4 secs.


In [22]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.00032299467058
PPV:  76.06060606060606
NPV:  86.83834048640917
Precision value of the model:  76.06060606060606
Accuracy of the model:  83.38192419825073
Recall of the model:  73.17784256559767
Specificity of the model:  88.48396501457727
F1 Score of the model:  74.59138187221399


In [23]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.709595
epoch: 0, train loss: 61.1399, lr: 0.000100, spent: 3.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.761765
epoch: 1, train loss: 55.1148, lr: 0.000100, spent: 7.8 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 84, test auc: 0.865236
EarlyStopping counter: 29 out of 30
epoch: 84, train loss: 18.2415, lr: 0.000100, spent: 344.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:36:02.464 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:36:02.485 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 85, test auc: 0.871068
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:36:02.721 | INFO     | transtab.trainer:train:137 - training complete, cost 348.6 secs.


In [24]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.08107166231757
PPV:  77.23076923076924
NPV:  86.93181818181817
Precision value of the model:  77.23076923076924
Accuracy of the model:  83.86783284742468
Recall of the model:  73.17784256559767
Specificity of the model:  89.21282798833819
F1 Score of the model:  75.1497005988024


In [25]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.731853
epoch: 0, train loss: 59.8836, lr: 0.000100, spent: 4.0 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.784470
epoch: 1, train loss: 53.8026, lr: 0.000100, spent: 8.0 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 70, test auc: 0.868792
EarlyStopping counter: 29 out of 30
epoch: 70, train loss: 23.6795, lr: 0.000100, spent: 287.4 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:40:54.353 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:40:54.379 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 71, test auc: 0.870948
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:40:54.616 | INFO     | transtab.trainer:train:137 - training complete, cost 291.5 secs.


In [26]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 86.60974593919201
PPV:  74.29467084639498
NPV:  85.07042253521126
Precision value of the model:  74.29467084639498
Accuracy of the model:  81.72983479105929
Recall of the model:  69.09620991253644
Specificity of the model:  88.0466472303207
F1 Score of the model:  71.6012084592145


In [27]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.694630
epoch: 0, train loss: 59.8287, lr: 0.000100, spent: 3.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.767843
epoch: 1, train loss: 56.3960, lr: 0.000100, spent: 8.0 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 48, test auc: 0.874557
EarlyStopping counter: 29 out of 30
epoch: 48, train loss: 28.4581, lr: 0.000100, spent: 196.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:44:15.211 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:44:15.233 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 49, test auc: 0.865830
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:44:15.513 | INFO     | transtab.trainer:train:137 - training complete, cost 200.5 secs.


In [28]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.8983331775026
PPV:  75.57471264367817
NPV:  88.2525697503671
Precision value of the model:  75.57471264367817
Accuracy of the model:  83.96501457725948
Recall of the model:  76.67638483965014
Specificity of the model:  87.60932944606414
F1 Score of the model:  76.12156295224314


### Raw Data

In [29]:
df=pd.read_csv('/content/delirium_icu_tr_mimic_.csv')
num_cols,cat_cols,bin_cols=getCols(df)
dataset_config={'bin': bin_cols,'cat':cat_cols,'num':num_cols}
allset, trainset, valset, cat_cols, num_cols, bin_cols= load_data(df,dataset_config=dataset_config)

['admission type', 'admission location', 'insurance', 'language', 'religion', 'marital status', 'ethnicity', 'gender']
[]
['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'gcs eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood)', 'hemoglobin', 'high respiratory rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak inspiratory pressure', 'phosphorous', 'plateau pressure', 'platelet count', 'potassiu

In [30]:
test=pd.read_csv('/content/delirium_icu_ts_mimic_.csv')
testset=load_test_data(test,dataset_config=dataset_config)

['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'gcs eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood)', 'hemoglobin', 'high respiratory rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak inspiratory pressure', 'phosphorous', 'plateau pressure', 'platelet count', 'potassium (whole blood)', 'pressure support', 'prothrombin time', 'resp rate (spont)', 'respiratory rate', 'respiratory rate set',

In [31]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.740821
epoch: 0, train loss: 59.4217, lr: 0.000100, spent: 4.5 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.778805
epoch: 1, train loss: 52.3567, lr: 0.000100, spent: 9.3 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 59, test auc: 0.859885
EarlyStopping counter: 29 out of 30
epoch: 59, train loss: 22.4023, lr: 0.000100, spent: 210.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:48:50.029 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:48:50.050 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 60, test auc: 0.871248
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:48:50.294 | INFO     | transtab.trainer:train:137 - training complete, cost 214.6 secs.


In [32]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.65098725871023
PPV:  70.47146401985111
NPV:  90.57507987220448
Precision value of the model:  70.47146401985111
Accuracy of the model:  82.70165208940719
Recall of the model:  82.79883381924198
Specificity of the model:  82.6530612244898
F1 Score of the model:  76.13941018766756


In [33]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.715287
epoch: 0, train loss: 59.7899, lr: 0.000100, spent: 3.5 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.776023
epoch: 1, train loss: 55.3624, lr: 0.000100, spent: 6.9 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 54, test auc: 0.875858
EarlyStopping counter: 29 out of 30
epoch: 54, train loss: 24.3665, lr: 0.000100, spent: 192.5 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:52:06.553 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:52:06.573 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 55, test auc: 0.867585
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:52:06.820 | INFO     | transtab.trainer:train:137 - training complete, cost 196.1 secs.


In [34]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 89.34330083553621
PPV:  77.68115942028986
NPV:  89.03508771929825
Precision value of the model:  77.68115942028986
Accuracy of the model:  85.22837706511176
Recall of the model:  78.134110787172
Specificity of the model:  88.77551020408163
F1 Score of the model:  77.90697674418605


In [35]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.700948
epoch: 0, train loss: 61.4368, lr: 0.000100, spent: 3.6 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.766675
epoch: 1, train loss: 57.3453, lr: 0.000100, spent: 7.0 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 51, test auc: 0.866023
EarlyStopping counter: 29 out of 30
epoch: 51, train loss: 25.1149, lr: 0.000100, spent: 181.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:55:11.899 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:55:11.922 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 52, test auc: 0.855335
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:55:12.162 | INFO     | transtab.trainer:train:137 - training complete, cost 185.0 secs.


In [36]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.86603371044377
PPV:  74.35897435897436
NPV:  87.90560471976401
Precision value of the model:  74.35897435897436
Accuracy of the model:  83.28474246841594
Recall of the model:  76.0932944606414
Specificity of the model:  86.88046647230321
F1 Score of the model:  75.21613832853026


In [37]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.713602
epoch: 0, train loss: 60.6887, lr: 0.000100, spent: 3.4 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.756734
epoch: 1, train loss: 56.2113, lr: 0.000100, spent: 7.2 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 77, test auc: 0.846321
EarlyStopping counter: 29 out of 30
epoch: 77, train loss: 17.7649, lr: 0.000100, spent: 272.4 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 10:59:48.393 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 10:59:48.416 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 78, test auc: 0.858157
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 10:59:48.664 | INFO     | transtab.trainer:train:137 - training complete, cost 276.1 secs.


In [38]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 87.67945328902074
PPV:  77.24358974358975
NPV:  85.77405857740585
Precision value of the model:  77.24358974358975
Accuracy of the model:  83.18756073858114
Recall of the model:  70.26239067055393
Specificity of the model:  89.65014577259474
F1 Score of the model:  73.58778625954197


In [39]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.734929
epoch: 0, train loss: 59.9118, lr: 0.000100, spent: 3.4 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.788060
epoch: 1, train loss: 53.5804, lr: 0.000100, spent: 6.9 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 45, test auc: 0.865950
EarlyStopping counter: 29 out of 30
epoch: 45, train loss: 27.9584, lr: 0.000100, spent: 161.6 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-17 11:02:33.990 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-17 11:02:34.010 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 46, test auc: 0.865590
EarlyStopping counter: 30 out of 30
early stopped


2022-11-17 11:02:34.260 | INFO     | transtab.trainer:train:137 - training complete, cost 165.2 secs.


In [40]:
# MIMIC raw data
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 88.4282909331996
PPV:  80.14440433212997
NPV:  83.9095744680851
Precision value of the model:  80.14440433212997
Accuracy of the model:  82.89601554907678
Recall of the model:  64.72303206997084
Specificity of the model:  91.98250728862973
F1 Score of the model:  71.61290322580645


### Data Encoded and Scaled

In [ ]:
df=pd.read_csv('/content/delirium_mimic_icu_train_imputed_processed.csv')
num_cols,cat_cols,bin_cols=getCols(df)
dataset_config={'bin': bin_cols,'cat':cat_cols,'num':num_cols}
allset, trainset, valset, cat_cols, num_cols, bin_cols= load_data(df,dataset_config=dataset_config)

['admission_type', 'admission_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity']
['gender']
['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'art.ph', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2(calc)', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'daily weight', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood - calc)', 'hemoglobin', 'high resp. rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'nbp mean', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak insp. pressure', 'phosphorous', 'plateau press

In [ ]:
test=pd.read_csv('/content/delirium_mimic_icu_test_imputed_processed.csv')
testset=load_test_data(test,dataset_config=dataset_config)

['alt', 'ast', 'admission weight (kg)', 'albumin', 'anion gap', 'art.ph', 'arterial base excess', 'arterial blood pressure mean', 'arterial co2(calc)', 'arterial paco2', 'arterial ph', 'bun', 'braden score', 'calcium non-ionized', 'central venous pressure', 'chloride (serum)', 'creatinine', 'daily weight', 'differential-basos', 'differential-eos', 'differential-lymphs', 'differential-monos', 'eye opening', 'fio2 set', 'fingerstick glucose', 'gcs total', 'hco3 (serum)', 'heart rate', 'hematocrit (whole blood - calc)', 'hemoglobin', 'high resp. rate', 'inr', 'inspired o2 fraction', 'ionized calcium', 'ldh', 'lactic acid', 'low exhaled min vol', 'magnesium', 'mean airway pressure', 'minute volume', 'motor response', 'nbp mean', 'non invasive blood pressure mean', 'o2 flow', 'o2 saturation pulseoxymetry', 'peep', 'ptt', 'peak insp. pressure', 'phosphorous', 'plateau pressure', 'platelet count', 'potassium (whole blood)', 'pressure support', 'previous weight (kg)', 'prothrombin time', 'resp

In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':1e-4,
    'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.713598
epoch: 0, train loss: 64.8773, lr: 0.000100, spent: 3.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.725115
epoch: 1, train loss: 61.1998, lr: 0.000100, spent: 7.9 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 60, test auc: 0.908678
EarlyStopping counter: 29 out of 30
epoch: 60, train loss: 25.2031, lr: 0.000100, spent: 240.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-13 19:51:38.856 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-13 19:51:38.873 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 61, test auc: 0.904568
EarlyStopping counter: 30 out of 30
early stopped


2022-11-13 19:51:39.116 | INFO     | transtab.trainer:train:137 - training complete, cost 244.8 secs.


In [ ]:
# MIMIC encoded and scaled
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 64.05579307941419
PPV:  64.36781609195403
NPV:  72.98245614035088
Precision value of the model:  64.36781609195403
Accuracy of the model:  71.52575315840622
Recall of the model:  32.6530612244898
Specificity of the model:  90.96209912536443
F1 Score of the model:  43.32688588007736


In [ ]:
# MIMIC encoded and scaled
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 67.06856836862191
PPV:  55.27950310559007
NPV:  70.73732718894009
Precision value of the model:  55.27950310559007
Accuracy of the model:  68.31875607385811
Recall of the model:  25.94752186588921
Specificity of the model:  89.50437317784257
F1 Score of the model:  35.317460317460316


In [ ]:
# MIMIC encoded and scaled
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 54.948618347797265
PPV:  43.459915611814345
NPV:  69.6969696969697
Precision value of the model:  43.459915611814345
Accuracy of the model:  63.65403304178814
Recall of the model:  30.029154518950435
Specificity of the model:  80.466472303207
F1 Score of the model:  35.51724137931034


In [ ]:
# MIMIC encoded and scaled
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 66.83482222543327
PPV:  50.29585798816568
NPV:  74.96382054992765
Precision value of the model:  50.29585798816568
Accuracy of the model:  66.86103012633625
Recall of the model:  49.56268221574344
Specificity of the model:  75.51020408163265
F1 Score of the model:  49.9265785609398


In [ ]:
# MIMIC encoded and scaled
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1].values,ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 68.88286343275337
PPV:  37.09884467265726
NPV:  78.4
Precision value of the model:  37.09884467265726
Accuracy of the model:  47.13313896987366
Recall of the model:  84.25655976676384
Specificity of the model:  28.57142857142857
F1 Score of the model:  51.515151515151516


# AUBMC Dataset

## Encoded Data

In [ ]:
aubmc_train=pd.read_csv('/content/aubmc_train_downsampled_enc.csv')
aubmc_test=pd.read_csv('/content/aubmc_test_enc.csv')

In [ ]:
aubmc_train.head()

,Unnamed: 0,BP_SYSTOLIC,BP_DIASTOLIC,TEMPERATURE,PULSE,WEIGHT,HEIGHT,RESPIRATIONS,BMI,BSA,...,ocd,anxiety,bipolar,abuse,antipsychotics,lithium,cancer,LOS,Age,Class
0,0,95.0,62.0,98.1,55.0,70.000403,1.799996,20.0,21.61,1.87,...,0,0,0,0,0,0,0,3.0,61.0,1
1,8,145.0,65.0,98.4,71.0,74.000587,1.689989,18.0,25.91,1.86,...,0,0,0,0,0,0,0,9.0,81.6,1
2,11,135.0,82.0,98.2,76.0,65.000313,1.659992,16.0,23.59,1.73,...,0,0,0,0,0,0,0,10.0,81.4,1
3,16,112.0,46.0,97.7,57.0,63.000504,1.699997,20.0,21.80,1.72,...,0,0,0,0,0,0,0,9.0,83.9,1
4,27,57.0,42.0,97.2,63.0,69.000498,1.499997,0.0,30.67,1.70,...,0,0,0,0,0,0,0,35.0,87.5,1


In [ ]:
aubmc_train.drop(['Unnamed: 0'],axis=1,inplace=True)
aubmc_test.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
num_cols,cat_cols,bin_cols=getCols(aubmc_train)

In [ ]:
dataset_config={'bin': bin_cols,'cat':cat_cols,'num':num_cols}
allset, trainset, valset, cat_cols, num_cols, bin_cols= load_data(aubmc_train,dataset_config=dataset_config)
testset=load_test_data(aubmc_test,dataset_config=dataset_config)

['325', '2401', '2405', '9637', '10608', '169448', '169449', '175233', '182878', '40802017']
['8428', '9639', '112703', '169861', '186252', '192673', '400051', '400562', '400580', 'cigarettes_yn', 'pipes_yn', 'cigars_yn', 'nationality', 'sex_c', 'rest_due_to_violence', 'use_of_rest', 'agitation', 'parkinson', 'alzeihmer', 'depression', 'ocd', 'anxiety', 'bipolar', 'abuse', 'antipsychotics', 'lithium', 'cancer']
['bp_systolic', 'bp_diastolic', 'temperature', 'pulse', 'weight', 'height', 'respirations', 'bmi', 'bsa', 'tobacco_type', 'cigarettes_yn', 'pipes_yn', 'cigars_yn', 'alcohol_use', 'add_line_1', 'nationality', 'sex_c', 'los', 'age']
# data: 552, # feat: 51, # cate: 10,  # bin: 27, # numerical: 19, pos rate: 0.33
['bp_systolic', 'bp_diastolic', 'temperature', 'pulse', 'weight', 'height', 'respirations', 'bmi', 'bsa', '325', '2401', '2405', '8428', '9637', '9639', '10608', '112703', '169448', '169449', '169861', '175233', '182878', '186252', '192673', '400051', '400562', '400580', '

In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.713589
epoch: 0, train loss: 8.9412, lr: 0.000050, spent: 1.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.695195
EarlyStopping counter: 1 out of 30
epoch: 1, train loss: 8.1462, lr: 0.000050, spent: 2.3 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 67, test auc: 0.780030
EarlyStopping counter: 29 out of 30
epoch: 67, train loss: 3.7018, lr: 0.000050, spent: 80.4 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:59:07.698 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:59:07.720 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 68, test auc: 0.745495
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:59:08.099 | INFO     | transtab.trainer:train:137 - training complete, cost 81.9 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.1639837520686
PPV:  38.028169014084504
NPV:  92.8030303030303
Precision value of the model:  38.028169014084504
Accuracy of the model:  81.19402985074626
Recall of the model:  58.69565217391305
Specificity of the model:  84.77508650519032
F1 Score of the model:  46.15384615384615


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.695946
epoch: 0, train loss: 8.8389, lr: 0.000050, spent: 1.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.692568
EarlyStopping counter: 1 out of 30
epoch: 1, train loss: 8.2456, lr: 0.000050, spent: 2.2 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 76, test auc: 0.750000
EarlyStopping counter: 29 out of 30
epoch: 76, train loss: 3.2294, lr: 0.000050, spent: 74.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:00:24.101 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:00:24.125 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 77, test auc: 0.758258
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:00:24.367 | INFO     | transtab.trainer:train:137 - training complete, cost 75.8 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.44230479915751
PPV:  33.70786516853933
NPV:  93.4959349593496
Precision value of the model:  33.70786516853933
Accuracy of the model:  77.61194029850746
Recall of the model:  65.21739130434783
Specificity of the model:  79.58477508650519
F1 Score of the model:  44.44444444444445


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.519895
epoch: 0, train loss: 9.5750, lr: 0.000050, spent: 0.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.647523
epoch: 1, train loss: 8.8771, lr: 0.000050, spent: 1.5 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 47, test auc: 0.750375
EarlyStopping counter: 29 out of 30
epoch: 47, train loss: 4.6775, lr: 0.000050, spent: 39.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:01:04.801 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:01:04.825 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 48, test auc: 0.763514
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:01:05.071 | INFO     | transtab.trainer:train:137 - training complete, cost 40.4 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 78.47149089814954
PPV:  42.5
NPV:  90.16949152542372
Precision value of the model:  42.5
Accuracy of the model:  84.4776119402985
Recall of the model:  36.95652173913043
Specificity of the model:  92.04152249134948
F1 Score of the model:  39.53488372093023


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.704204
epoch: 0, train loss: 8.9124, lr: 0.000050, spent: 1.1 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.706081
epoch: 1, train loss: 8.5749, lr: 0.000050, spent: 2.4 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 46, test auc: 0.753003
EarlyStopping counter: 29 out of 30
epoch: 46, train loss: 4.9790, lr: 0.000050, spent: 40.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:01:46.646 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:01:46.678 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 47, test auc: 0.759009
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:01:47.060 | INFO     | transtab.trainer:train:137 - training complete, cost 41.7 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 77.1776741387092
PPV:  34.21052631578947
NPV:  92.27799227799228
Precision value of the model:  34.21052631578947
Accuracy of the model:  79.1044776119403
Recall of the model:  56.52173913043478
Specificity of the model:  82.69896193771626
F1 Score of the model:  42.62295081967213


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.695946
epoch: 0, train loss: 9.6253, lr: 0.000050, spent: 0.8 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.705706
epoch: 1, train loss: 8.7301, lr: 0.000050, spent: 1.4 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 58, test auc: 0.733859
EarlyStopping counter: 29 out of 30
epoch: 58, train loss: 4.0547, lr: 0.000050, spent: 40.5 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:03:56.417 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:03:56.442 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 59, test auc: 0.735360
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:03:56.692 | INFO     | transtab.trainer:train:137 - training complete, cost 41.4 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.11087708740786
PPV:  40.0
NPV:  92.5925925925926
Precision value of the model:  40.0
Accuracy of the model:  82.38805970149254
Recall of the model:  56.52173913043478
Specificity of the model:  86.50519031141869
F1 Score of the model:  46.846846846846844


## Raw Data

In [ ]:
aubmc_train=pd.read_csv('/content/aubmc_train_downsampled.csv')
aubmc_test=pd.read_csv('/content/aubmc_test.csv')

In [ ]:
aubmc_train.head()

,Unnamed: 0,BP_SYSTOLIC,BP_DIASTOLIC,TEMPERATURE,PULSE,WEIGHT,HEIGHT,RESPIRATIONS,BMI,BSA,...,anxiety,bipolar,abuse,antipsychotics,lithium,cancer,LOS,Age,Classes,Class
0,0,95.0,62.0,98.1,55.0,70.000403,1.799996,20.0,21.61,1.87,...,0,0,0,0,0,0,3.0,61.0,possible,1
1,8,145.0,65.0,98.4,71.0,74.000587,1.689989,18.0,25.91,1.86,...,0,0,0,0,0,0,9.0,81.6,possible,1
2,11,135.0,82.0,98.2,76.0,65.000313,1.659992,16.0,23.59,1.73,...,0,0,0,0,0,0,10.0,81.4,confirmed,1
3,16,112.0,46.0,97.7,57.0,63.000504,1.699997,20.0,21.80,1.72,...,0,0,0,0,0,0,9.0,83.9,possible,1
4,27,57.0,42.0,97.2,63.0,69.000498,1.499997,0.0,30.67,1.70,...,0,0,0,0,0,0,35.0,87.5,possible,1


In [ ]:
aubmc_train.drop(['Unnamed: 0','Classes'],axis=1,inplace=True)
aubmc_test.drop(['Unnamed: 0','Classes'],axis=1,inplace=True)

In [ ]:
aubmc_train.head()

,BP_SYSTOLIC,BP_DIASTOLIC,TEMPERATURE,PULSE,WEIGHT,HEIGHT,RESPIRATIONS,BMI,BSA,325,...,ocd,anxiety,bipolar,abuse,antipsychotics,lithium,cancer,LOS,Age,Class
0,95.0,62.0,98.1,55.0,70.000403,1.799996,20.0,21.61,1.87,0,...,0,0,0,0,0,0,0,3.0,61.0,1
1,145.0,65.0,98.4,71.0,74.000587,1.689989,18.0,25.91,1.86,0,...,0,0,0,0,0,0,0,9.0,81.6,1
2,135.0,82.0,98.2,76.0,65.000313,1.659992,16.0,23.59,1.73,0,...,0,0,0,0,0,0,0,10.0,81.4,1
3,112.0,46.0,97.7,57.0,63.000504,1.699997,20.0,21.80,1.72,0,...,0,0,0,0,0,0,0,9.0,83.9,1
4,57.0,42.0,97.2,63.0,69.000498,1.499997,0.0,30.67,1.70,0,...,0,0,0,0,0,0,0,35.0,87.5,1


In [ ]:
num_cols,cat_cols,bin_cols=getCols(aubmc_train)

In [ ]:
#raw
dataset_config={'bin': bin_cols,'cat':cat_cols,'num':num_cols}
allset, trainset, valset, cat_cols, num_cols, bin_cols= load_data(aubmc_train,dataset_config=dataset_config)
testset=load_test_data(aubmc_test,dataset_config=dataset_config)

['325', '2401', '2405', '9637', '10608', '169448', '169449', '175233', '182878', '40802017', 'tobacco_type', 'cigarettes_yn', 'pipes_yn', 'cigars_yn', 'alcohol_use', 'add_line_1', 'nationality', 'sex_c']
['8428', '9639', '112703', '169861', '186252', '192673', '400051', '400562', '400580', 'rest_due_to_violence', 'use_of_rest', 'agitation', 'parkinson', 'alzeihmer', 'depression', 'ocd', 'anxiety', 'bipolar', 'abuse', 'antipsychotics', 'lithium', 'cancer']
['bp_systolic', 'bp_diastolic', 'temperature', 'pulse', 'weight', 'height', 'respirations', 'bmi', 'bsa', 'los', 'age']
# data: 552, # feat: 51, # cate: 18,  # bin: 22, # numerical: 11, pos rate: 0.33
['bp_systolic', 'bp_diastolic', 'temperature', 'pulse', 'weight', 'height', 'respirations', 'bmi', 'bsa', '325', '2401', '2405', '8428', '9637', '9639', '10608', '112703', '169448', '169449', '169861', '175233', '182878', '186252', '192673', '400051', '400562', '400580', '40802017', 'tobacco_type', 'cigarettes_yn', 'pipes_yn', 'cigars_yn

In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':5e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.678679
epoch: 0, train loss: 8.2371, lr: 0.000050, spent: 0.8 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 1, test auc: 0.698198
epoch: 1, train loss: 7.8850, lr: 0.000050, spent: 1.5 secs


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 56, test auc: 0.781156
EarlyStopping counter: 29 out of 30
epoch: 56, train loss: 1.2581, lr: 0.000050, spent: 44.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 21:41:33.521 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 21:41:33.541 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 57, test auc: 0.774775
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 21:41:33.789 | INFO     | transtab.trainer:train:137 - training complete, cost 45.6 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 81.9091319392207
PPV:  43.54838709677419
NPV:  93.04029304029304
Precision value of the model:  43.54838709677419
Accuracy of the model:  83.88059701492537
Recall of the model:  58.69565217391305
Specificity of the model:  87.88927335640139
F1 Score of the model:  50.0


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 83.16533774635172
PPV:  38.095238095238095
NPV:  94.42231075697211
Precision value of the model:  38.095238095238095
Accuracy of the model:  80.29850746268656
Recall of the model:  69.56521739130434
Specificity of the model:  82.00692041522491
F1 Score of the model:  49.230769230769226


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 77.85467128027682
PPV:  37.03703703703704
NPV:  93.7007874015748
Precision value of the model:  37.03703703703704
Accuracy of the model:  80.0
Recall of the model:  65.21739130434783
Specificity of the model:  82.35294117647058
F1 Score of the model:  47.24409448818897


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 84.22596660147435
PPV:  35.64356435643564
NPV:  95.72649572649573
Precision value of the model:  35.64356435643564
Accuracy of the model:  77.61194029850746
Recall of the model:  78.26086956521739
Specificity of the model:  77.50865051903114
F1 Score of the model:  48.97959183673469


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.32902061080188
PPV:  37.03703703703704
NPV:  93.7007874015748
Precision value of the model:  37.03703703703704
Accuracy of the model:  80.0
Recall of the model:  65.21739130434783
Specificity of the model:  82.35294117647058
F1 Score of the model:  47.24409448818897


## Transfer Learning

In [ ]:
!unzip '/content/pretrained_mimic.zip'

Archive:  /content/pretrained_mimic.zip
   creating: content/ckpt/pretrained_mimic/
  inflating: content/ckpt/pretrained_mimic/input_encoder.bin  
  inflating: content/ckpt/pretrained_mimic/pytorch_model.bin  
   creating: content/ckpt/pretrained_mimic/extractor/
   creating: content/ckpt/pretrained_mimic/extractor/tokenizer/
  inflating: content/ckpt/pretrained_mimic/extractor/tokenizer/special_tokens_map.json  
  inflating: content/ckpt/pretrained_mimic/extractor/tokenizer/vocab.txt  
  inflating: content/ckpt/pretrained_mimic/extractor/tokenizer/tokenizer_config.json  
  inflating: content/ckpt/pretrained_mimic/extractor/tokenizer/tokenizer.json  
  inflating: content/ckpt/pretrained_mimic/extractor/extractor.json  


## Encoded Data

In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 23:03:57.850 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 23:03:57.851 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 23:03:57.855 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 23:03:57.878 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.696321
epoch: 0, train loss: 8.8119, lr: 0.000020, spent: 0.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 47, test auc: 0.736486
EarlyStopping counter: 29 out of 30
epoch: 47, train loss: 5.0464, lr: 0.000020, spent: 34.0 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:04:32.521 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:04:32.540 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 48, test auc: 0.728979
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:04:32.786 | INFO     | transtab.trainer:train:137 - training complete, cost 34.9 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.01308861140365
PPV:  35.714285714285715
NPV:  92.0754716981132
Precision value of the model:  35.714285714285715
Accuracy of the model:  80.29850746268656
Recall of the model:  54.347826086956516
Specificity of the model:  84.42906574394463
F1 Score of the model:  43.103448275862064


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 23:04:33.058 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 23:04:33.059 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 23:04:33.060 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 23:04:33.083 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.698949
epoch: 0, train loss: 8.7760, lr: 0.000020, spent: 0.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 56, test auc: 0.732733
EarlyStopping counter: 29 out of 30
epoch: 56, train loss: 4.4012, lr: 0.000020, spent: 39.6 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:05:13.388 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:05:13.409 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 57, test auc: 0.746246
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:05:13.661 | INFO     | transtab.trainer:train:137 - training complete, cost 40.6 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.4874379419287
PPV:  45.83333333333333
NPV:  91.63763066202091
Precision value of the model:  45.83333333333333
Accuracy of the model:  85.07462686567165
Recall of the model:  47.82608695652174
Specificity of the model:  91.00346020761245
F1 Score of the model:  46.80851063829787


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 23:05:13.921 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 23:05:13.923 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 23:05:13.928 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 23:05:13.951 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.699324
epoch: 0, train loss: 8.6338, lr: 0.000020, spent: 1.0 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 47, test auc: 0.738739
EarlyStopping counter: 29 out of 30
epoch: 47, train loss: 5.0702, lr: 0.000020, spent: 34.6 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:05:49.419 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:05:49.440 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 48, test auc: 0.734234
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:05:49.687 | INFO     | transtab.trainer:train:137 - training complete, cost 35.7 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 78.86264480216639
PPV:  42.857142857142854
NPV:  91.25874125874127
Precision value of the model:  42.857142857142854
Accuracy of the model:  84.17910447761194
Recall of the model:  45.65217391304348
Specificity of the model:  90.3114186851211
F1 Score of the model:  44.21052631578947


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 23:05:49.958 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 23:05:49.963 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 23:05:49.967 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 23:05:49.989 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.702327
epoch: 0, train loss: 8.6725, lr: 0.000020, spent: 0.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 53, test auc: 0.753378
EarlyStopping counter: 29 out of 30
epoch: 53, train loss: 4.5963, lr: 0.000020, spent: 37.9 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:06:28.578 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:06:28.601 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 54, test auc: 0.725601
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:06:28.863 | INFO     | transtab.trainer:train:137 - training complete, cost 38.9 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.96088460959831
PPV:  48.78048780487805
NPV:  91.15646258503402
Precision value of the model:  48.78048780487805
Accuracy of the model:  85.97014925373134
Recall of the model:  43.47826086956522
Specificity of the model:  92.73356401384083
F1 Score of the model:  45.97701149425287


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 23:06:29.123 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 23:06:29.125 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 23:06:29.129 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 23:06:29.152 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.702327
epoch: 0, train loss: 8.5837, lr: 0.000020, spent: 0.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 61, test auc: 0.740991
EarlyStopping counter: 29 out of 30
epoch: 61, train loss: 3.8212, lr: 0.000020, spent: 41.7 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 23:07:11.516 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 23:07:11.538 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 62, test auc: 0.733483
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 23:07:11.782 | INFO     | transtab.trainer:train:137 - training complete, cost 42.6 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.50248232285242
PPV:  47.82608695652174
NPV:  91.6955017301038
Precision value of the model:  47.82608695652174
Accuracy of the model:  85.67164179104478
Recall of the model:  47.82608695652174
Specificity of the model:  91.6955017301038
F1 Score of the model:  47.82608695652174


## Raw Data

In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 22:38:41.553 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 22:38:41.560 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 22:38:41.565 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 22:38:41.594 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.655781
epoch: 0, train loss: 9.3168, lr: 0.000020, spent: 1.2 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 46, test auc: 0.724850
EarlyStopping counter: 29 out of 30
epoch: 46, train loss: 1.6351, lr: 0.000020, spent: 40.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:39:22.993 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:39:23.013 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 47, test auc: 0.738739
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:39:23.360 | INFO     | transtab.trainer:train:137 - training complete, cost 41.7 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.46487137054311
PPV:  39.130434782608695
NPV:  92.85714285714286
Precision value of the model:  39.130434782608695
Accuracy of the model:  81.7910447761194
Recall of the model:  58.69565217391305
Specificity of the model:  85.46712802768167
F1 Score of the model:  46.95652173913044


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 22:39:24.086 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 22:39:24.088 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 22:39:24.094 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 22:39:24.123 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.652778
epoch: 0, train loss: 9.2304, lr: 0.000020, spent: 1.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 46, test auc: 0.706832
EarlyStopping counter: 29 out of 30
epoch: 46, train loss: 1.6888, lr: 0.000020, spent: 44.6 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:40:09.413 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:40:09.434 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 47, test auc: 0.718844
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:40:09.673 | INFO     | transtab.trainer:train:137 - training complete, cost 45.5 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 80.26929441853467
PPV:  41.07142857142857
NPV:  91.75627240143369
Precision value of the model:  41.07142857142857
Accuracy of the model:  83.28358208955224
Recall of the model:  50.0
Specificity of the model:  88.58131487889274
F1 Score of the model:  45.09803921568628


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 22:40:09.938 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 22:40:09.940 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 22:40:09.943 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 22:40:09.969 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.657282
epoch: 0, train loss: 8.9333, lr: 0.000020, spent: 0.8 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 43, test auc: 0.736111
EarlyStopping counter: 29 out of 30
epoch: 43, train loss: 1.7104, lr: 0.000020, spent: 31.1 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:40:41.747 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:40:41.769 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 44, test auc: 0.746622
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:40:42.034 | INFO     | transtab.trainer:train:137 - training complete, cost 32.0 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.55468632465774
PPV:  42.857142857142854
NPV:  91.25874125874127
Precision value of the model:  42.857142857142854
Accuracy of the model:  84.17910447761194
Recall of the model:  45.65217391304348
Specificity of the model:  90.3114186851211
F1 Score of the model:  44.21052631578947


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 22:40:42.299 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 22:40:42.301 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 22:40:42.303 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 22:40:42.325 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.658033
epoch: 0, train loss: 8.9529, lr: 0.000020, spent: 0.8 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 45, test auc: 0.745871
EarlyStopping counter: 29 out of 30
epoch: 45, train loss: 1.5183, lr: 0.000020, spent: 33.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:41:16.337 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:41:16.357 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 46, test auc: 0.736111
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:41:16.612 | INFO     | transtab.trainer:train:137 - training complete, cost 34.3 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 79.87061832405597
PPV:  39.6551724137931
NPV:  91.69675090252709
Precision value of the model:  39.6551724137931
Accuracy of the model:  82.68656716417911
Recall of the model:  50.0
Specificity of the model:  87.88927335640139
F1 Score of the model:  44.230769230769226


In [ ]:
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)
model.load('/content/content/ckpt/pretrained_mimic')
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols})
training_arguments = {
    'num_epoch':100,
    'patience':30,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':32,
    'lr':2e-5,
    #'weight_decay':1e-4
    }
transtab.train(model, trainset, valset, **training_arguments)



2022-11-15 22:41:16.887 | INFO     | transtab.modeling_transtab:load:773 - missing keys: []
2022-11-15 22:41:16.888 | INFO     | transtab.modeling_transtab:load:774 - unexpected keys: []
2022-11-15 22:41:16.892 | INFO     | transtab.modeling_transtab:load:775 - load model from /content/content/ckpt/pretrained_mimic
2022-11-15 22:41:16.915 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from /content/content/ckpt/pretrained_mimic/extractor/extractor.json


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 0, test auc: 0.653529
epoch: 0, train loss: 9.1009, lr: 0.000020, spent: 0.8 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


...

/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)


epoch: 42, test auc: 0.743994
EarlyStopping counter: 29 out of 30
epoch: 42, train loss: 1.7780, lr: 0.000020, spent: 30.3 secs


/usr/local/lib/python3.7/dist-packages/transtab/trainer.py:165: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  y_test = pd.concat(y_test, 0)
2022-11-15 22:41:47.928 | INFO     | transtab.trainer:train:132 - load best at last from ./checkpoint
2022-11-15 22:41:47.945 | INFO     | transtab.trainer:save_model:239 - saving model checkpoint to ./checkpoint


epoch: 43, test auc: 0.728979
EarlyStopping counter: 30 out of 30
early stopped


2022-11-15 22:41:48.206 | INFO     | transtab.trainer:train:137 - training complete, cost 31.3 secs.


In [ ]:
ypred = transtab.predict(model, testset[0])
from sklearn.metrics import roc_auc_score
print('ROC_AUC: '+str(roc_auc_score(testset[1],ypred)*100))
pred = np.round(ypred)
from sklearn.metrics import log_loss, roc_curve,confusion_matrix, auc
cm=confusion_matrix(testset[1],pred)
metrics(cm)

ROC_AUC: 77.97502632766663
PPV:  36.61971830985916
NPV:  92.42424242424242
Precision value of the model:  36.61971830985916
Accuracy of the model:  80.59701492537313
Recall of the model:  56.52173913043478
Specificity of the model:  84.42906574394463
F1 Score of the model:  44.44444444444444
